# 4. Stitch outputs together
- Once you have fitted the weak, medium and dense diads, stitch them all together, and apply the Ne correction model

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import DiadFit as pf
pf.__version__

'0.0.57'

In [2]:
import os
DayFolder=os.getcwd()
meta_path=DayFolder +  '\MetaData'
spectra_path=DayFolder + '\Spectra'

## Load in specra, and combine

In [3]:
import os.path
from os import path
if path.exists('Discarded_df.xlsx'):
    discard=pd.read_excel('Discarded_df.xlsx')
else:
    discard=None
if path.exists('Weak_Diads.xlsx'):
    grp1=pd.read_excel('Weak_Diads.xlsx')
else:
    grp1=None
if path.exists('Medium_Diads.xlsx'):
    grp2=pd.read_excel('Medium_Diads.xlsx')
else:
    grp2=None
if path.exists('Strong_Diads.xlsx'):
    grp3=pd.read_excel('Strong_Diads.xlsx')
else:
    grp3=None
df2=pd.concat([grp1, grp2, grp3], axis=0)
if discard is not None:
    discard_cols=discard[discard.columns.intersection(df2.columns)]
    df2=pd.concat([df2, discard_cols])

## First, get the metadata for this day

In [4]:
# Put the common string you can in all your Ne lines here
ID_str='_'
file_ext_meta='txt' # Will only take files of this type 
diad_meta=pf.get_files(path=meta_path,
file_ext='txt', exclude_str=['N', 'FG','Cap','window'],
            exclude_type='.png',   sort=True)
diad_meta

['POC1.txt',
 'POC10.txt',
 'POC11.txt',
 'POC12.txt',
 'POC13.txt',
 'POC14.txt',
 'POC15.txt',
 'POC16.txt',
 'POC17.txt',
 'POC18.txt',
 'POC19.txt',
 'POC2.txt',
 'POC20.txt',
 'POC21.txt',
 'POC22.txt',
 'POC23.txt',
 'POC24.txt',
 'POC25.txt',
 'POC26.txt',
 'POC27.txt',
 'POC28.txt',
 'POC29.txt',
 'POC3.txt',
 'POC30.txt',
 'POC31.txt',
 'POC32.txt',
 'POC33.txt',
 'POC34.txt',
 'POC35.txt',
 'POC36.txt',
 'POC37.txt',
 'POC38.txt',
 'POC39.txt',
 'POC4.txt',
 'POC40.txt',
 'POC41.txt',
 'POC42.txt',
 'POC43.txt',
 'POC44.txt',
 'POC45.txt',
 'POC46.txt',
 'POC47.txt',
 'POC48.txt',
 'POC49.txt',
 'POC5.txt',
 'POC50.txt',
 'POC51.txt',
 'POC52.txt',
 'POC53.txt',
 'POC54.txt',
 'POC55.txt',
 'POC56.txt',
 'POC57-0.04.txt',
 'POC58.txt',
 'POC6.txt',
 'POC7.txt',
 'POC8.txt',
 'POC9.txt']

## Now get all the important information out of the metadata files

In [5]:
meta=pf.stitch_metadata_in_loop_witec(path=meta_path, 
Allfiles=diad_meta, prefix=False,
trupower=False)
meta['filename'].iloc[0]

100%|██████████████████████████████████████████████████████████████████████████████████| 58/58 [00:01<00:00, 31.61it/s]

Done
Oof, try not to work after midnight!


'POC49.txt'

## Then get a simple file name you can stitch with the spectra

In [6]:
file_m=pf.extracting_filenames_generic(names=meta['filename'],
    prefix=False, str_prefix=" ",
   file_type='.txt')
# Checks they are all unique
file_m[0]

g:\my drive\berkeley_new\diadfit_outer\src\DiadFit\importing_data_files.py:1065: FutureWarning: The default value of regex will change from True to False in a future version.
  names=names.str.replace(file_type, '')


'POC49'

In [7]:
type(file_m)

list

## Now get filename from spectra into same form as metadata

In [8]:
type(df2['filename'].reset_index(drop=True))

pandas.core.series.Series

In [10]:
# Remove these to get the pure file name
file_s=pf.extracting_filenames_generic(
    prefix=False, str_prefix=" ",
    suffix=True, str_suffix='_CRR_DiadFit',
    names=df2['filename'].reset_index(drop=True),
   file_type='.txt')

file_s[0]

'Cap19-05X2'

In [10]:
# Combining them together
meta['name_for_matching']=file_m
df2['name_for_matching']=file_s
df_combo=df2.merge(meta, on='name_for_matching')

In [11]:
from pickle import load
import pickle
import os
pickle_path=os.getcwd()
file_pkl=pickle_path + '/' + 'Ne_Line_Regression_Example_cali.pck'
with open(file_pkl, 'rb') as file_handle:
    my_loaded_model = pickle.load(file_handle)

In [12]:
df_combo['Ne_Correction']=my_loaded_model(df_combo['sec since midnight'])
df_combo.insert(1,'Corrected_Splitting', df_combo['Ne_Correction']*df_combo['Splitting'])

In [14]:
df_combo.to_excel('Calibration_fitting.xlsx')

In [15]:
df_combo

,Unnamed: 0,Corrected_Splitting,filename_x,Splitting,Diad1_Combofit_Cent,Diad1_Combofit_Height,Diad1_Voigt_Cent,Diad1_Voigt_Area,Diad1_Voigt_Sigma,Diad1_Residual,...,Day,power (mW),Int_time (s),accumulations,Mag (X),duration,24hr_time,sec since midnight,Spectral Center,Ne_Correction
0,0,102.803070,POC51_CRR_DiadFit,102.740281,1286.697274,179.754105,1286.697274,235.084296,0.614302,3.138013,...,29.0,NaN,60.0,3.0,5.0,"['0h', '3m', '2s']",12:17:28 AM\n,87448.0,1341.0,1.000611
1,0,102.797137,POC52_CRR_DiadFit,102.734244,1286.689828,244.740999,1286.689828,311.318690,0.596053,3.751448,...,29.0,NaN,90.0,3.0,5.0,"['0h', '4m', '31s']",12:31:31 AM\n,88291.0,1341.0,1.000612
2,0,102.823766,POC53_CRR_DiadFit,102.760795,1286.675369,219.447559,1286.675369,282.357216,0.604368,2.948802,...,29.0,NaN,90.0,3.0,5.0,"['0h', '4m', '32s']",12:39:13 AM\n,88753.0,1341.0,1.000613
3,0,102.822361,POC54_CRR_DiadFit,102.759329,1286.683797,183.617520,1286.683847,240.599134,0.615497,3.126455,...,29.0,NaN,90.0,3.0,5.0,"['0h', '4m', '32s']",12:46:57 AM\n,89217.0,1341.0,1.000613
4,0,102.790498,POC55,102.727423,1286.707961,156.339859,1286.707961,206.275442,0.619749,3.257327,...,29.0,NaN,90.0,3.0,5.0,"['0h', '4m', '32s']",12:54:26 AM\n,89666.0,1341.0,1.000614
5,0,102.792960,POC56_CRR_DiadFit,102.729820,1286.713944,131.045569,1286.713994,171.469139,0.614621,3.118194,...,29.0,NaN,90.0,3.0,5.0,"['0h', '4m', '32s']",1:01:55 AM\n,90115.0,1341.0,1.000615
6,0,102.821422,POC57-0.04,102.758188,1286.687409,126.454482,1286.687409,158.141870,0.587422,2.955460,...,29.0,NaN,90.0,5.0,5.0,"['0h', '7m', '32s']",1:10:45 AM\n,90645.0,1341.0,1.000615
7,0,102.791145,POC58_CRR_DiadFit,102.727840,1286.718187,118.641136,1286.718187,150.653458,0.596460,2.920535,...,29.0,NaN,90.0,5.0,5.0,"['0h', '7m', '32s']",1:20:58 AM\n,91258.0,1341.0,1.000616
8,0,102.869748,POC39_CRR_DiadFit,102.807542,1286.514202,2008.914189,1286.514252,3122.915531,0.652655,4.204885,...,28.0,NaN,45.0,3.0,5.0,"['0h', '2m', '16s']",10:41:24 PM\n,81684.0,1341.0,1.000605
9,0,102.866307,POC40_CRR_DiadFit,102.804069,1286.536041,1867.371704,1286.536091,2830.026406,0.644190,4.130815,...,28.0,NaN,45.0,3.0,5.0,"['0h', '2m', '16s']",10:47:37 PM\n,82057.0,1341.0,1.000605
